<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 4 - West Nile Virus Prediction

<div style="border: 1px solid thistle; padding: 5px 20px 10px 20px;">
    <h2><font color = "77658f" style=";">Feedback</font></h2><br>
    <font color = "77658f"> • No issues with data cleaning. For EDA, more visualizations of weather features and number of mosquitoes/ wmv cases against time could be included for analysis, discussion and selection of features. <br> • Demonstrated proficiency in python <br> • For features such as day, month, you can consider modelling them as cyclical features. Do readup the following post. <br> http://blog.davidkaleko.com/feature-engineering-cyclical-features.html  <br> • It is encouraging to see that the team's approach included the clustering of traps. A good number of models were built and inference were conducted on the final model. However, do note that smote was not used within the pipeline for the models. This would mean that during the gridsearch, the synthetic data points created would also be in the validation set within each fold of cv and this is undesirable. By including smote in the pipeline, the k-neighbors hyperparamter could also be tuned. <br> • Presentation - Slides were clean and nice. Could have used a flow chart instead of a table for  <br> • Presentation - Slides were clean and nice. Could have used a flow chart instead of a table for results slide. Coule explore lagging the weather features by more time periods. Color could be darker for data points on maps.  <br><br>
        • Organization: <b>2</b> <br>
        • Data Structures: <b>3</b> <br>
        • Python Syntax and Control Flow: <b>3</b> <br>
        • Probability and Statistics: <b>2</b><br>
        • Modelling: <b>2</b><br>
        • Presentation: <b>2</b><br>
    </font>
</div>

# Part 1 : Introduction


## Contents:
* [Organisation of Notebooks](#Organisation-of-Notebooks)
* [Problem Statement](#Problem-Statement)
* [Background](#Background)
* [Executive Summary](#Executive-Summary)
* [Data Dictionary](#Data-Dictionary)

## Organisation of Notebooks:
1. Introduction
2. [EDA](./02_EDA.ipynb)
3. [Model Evaluation](./03_Model_Evaluation.ipynb)
4. [Recommendation and Conclusion](./04_Recommendation_and_Conclusion.ipynb)

## Problem Statement

Our team, as part of the Chicago Department of Public Health (CDPH), has been tasked with predicting the next outbreak of the West Nile Virus (WNV) in Chicago. This information would be useful for the City of Chicago and the CDPH to identify areas of high risk of outbreak in advance and to allocate resources accordingly. We will also perform a cost-benefit analysis of the current vector control program and make recommendation on the next steps going forward.

## Background

West Nile Virus emerged in the United States in the New York metropolitan area in the fall of 1999. Since then, the virus, which can be transmitted to humans by the bite of an infected mosquito, has quickly spread across the country.

In Illinois, West Nile Virus was first identified in September 2001 when laboratory tests confirmed its presence in two dead crows found in the Chicago area. The following year, the state's first human cases and deaths from West Nile disease were recorded and all but two of the state's 102 counties eventually reported a positive human, bird, mosquito or horse. By the end of 2002, Illinois had counted more human cases (884) and deaths (64) than any other state in the United States.([*source*](https://dph.illinois.gov/topics-services/diseases-and-conditions/west-nile-virus))

By 2004, the City of Chicago and the Chicago Department of Public Health (CDPH) had established a comprehensive surveillance and control program that is still in effect today. Every week from June to September, mosquitos in traps across the city are tested for the virus. The results of these tests influence when and where the city will spray airborne pesticides to control adult mosquito populations. ([*source*](https://www.kaggle.com/c/predict-west-nile-virus))

About 25% of those became infected become ill with mild, influenza-like symptoms. There is however a 1 in 230 chance that someone infected develop severe symptoms which affect the central nervous system. ([*source*](https://www.uptodate.com/contents/west-nile-virus-infection-beyond-the-basics))

## Executive Summary

For this project, datasets containing information such as mosquitos species caught, number of mosquitoes caught, presence of WNV in the mosquitos and coordinate location of traps were used. Information on the date and location of spraying and various weather parameters were also utilized. The following summarises the analysis process:

1. Impute or remove missing or duplicated data from the train, test, weather and spray datasets

2. Merge the train and spray datasets and perform EDA on the effect of spraying on presence of WNV and number of mosquitos

3. Merge the train and weather datasets and perform feature engineering and selection by evaluating the ROC_AUC score of models against the selected features iteratively 

4. Tune the hyperparameters for 18 models by gridsearching and choose the best one based on ROC_AUC score on the train set, the CV ROC_AUC score on the train set , the ROC_AUC score on the test set and finally the Kaggle score

The best model selected was the XGBoost Classifier (Model 18) as even though it has a lower ROC_AUC than the Logistic Regression (Model 17), it has a higher sensitivity which is important because Chicago city officials might make expensive decisions on mosquito-eradication methods (spray in areas we predict to be positive, and don't spray in areas we predict to be negative). Therefore it is also important to have as high true positive as possible, and as low false negative as possible. This points to using sensitivity as a secondary metric for us to use as assessment since ROC_AUC is quite close between Models 17 and 18.

The following table summarizes the performance of all 18 models:

| Model No. | Classifier | CV Score (train) | ROC_AUC (train) | ROC_AUC (test) | Kaggle Score | Runtime (sec) |
|---|---|---|---|---|---|---|
| 1 | LogisticRegression(random_state=42, solver='liblinear') | 0.989000 | 0.994000 | 0.830000 | 0.684000 | 7 |
| 2 | KNeighborsClassifier() | 0.919000 | 1.000000 | 0.7776000 | 0.596000 | 24 |
| 4 | RandomForestClassifier(random_state=42) | 0.979000 | 0.996000 | 0.804000 | 0.643000 | 33 |
| 4 | ExtraTreesClassifier(random_state=42) | 0.989000 | 1.000000 | 0.809000 | 0.710000 | 93 |
| 5 | SVC(max_iter=10000, random_state=42) | 0.973000 | 0.981000 | 0.803000 | 0.598000 | 136 |
| 6 | XGBClassifier(base_score=None, booster=None, colsample_bylevel=None, <br>colsample_bynode=None, colsample_bytree=None, gamma=None, gpu_id=None, <br>importance_type='gain', interaction_constraints=None, learning_rate=None, <br>max_delta_step=None, max_depth=None, min_child_weight=None, missing=nan, <br>monotone_constraints=None, n_estimators=100, n_jobs=None, num_parallel_tree=None, <br>random_state=42, reg_alpha=None, reg_lambda=None, scale_pos_weight=None, <br>subsample=None, tree_method=None, validate_parameters=None, verbosity=None) | 0.992000 | 1.000000 | 0.831000 | 0.697000 | 133 |
| 7 | LogisticRegression(random_state=42) | 0.989000 | 0.994000 | 0.828000 | 0.684000 | 176 |
| 8 | XGBClassifier(base_score=None, booster=None, colsample_bylevel=None, <br>colsample_bynode=None, colsample_bytree=None, gamma=None, gpu_id=None, <br>importance_type='gain', interaction_constraints=None, learning_rate=None, <br>max_delta_step=None, max_depth=None, min_child_weight=None, missing=nan, <br>monotone_constraints=None, n_estimators=100, n_jobs=None, num_parallel_tree=None, <br>random_state=42, reg_alpha=None, reg_lambda=None, scale_pos_weight=None, <br>subsample=None, tree_method=None, validate_parameters=None, verbosity=None) | 0.991000 | 0.999000 | 0.830000 | 0.658000 | 104 |
| 9 | LogisticRegression(random_state=42, solver='liblinear') | 0.989000 | 0.994000 | 0.826000 | 0.693000 | 10 |
| 10 | XGBClassifier(base_score=None, booster=None, colsample_bylevel=None, <br>colsample_bynode=None, colsample_bytree=None, gamma=None, gpu_id=None, <br>importance_type='gain', interaction_constraints=None, learning_rate=None, <br>max_delta_step=None, max_depth=None, min_child_weight=None, missing=nan, <br>monotone_constraints=None, n_estimators=100, n_jobs=None, num_parallel_tree=None, <br>random_state=42, reg_alpha=None, reg_lambda=None, scale_pos_weight=None, subsample=None, <br>tree_method=None, validate_parameters=None, verbosity=None) | 0.990000 | 1.000000 | 0.797000 | 0.673000 | 182 |
| 11 | LogisticRegression(random_state=42, solver='liblinear') | 0.988000 | 0.993000 | 0.832000 | 0.717000 | 6 |
| 12 | XGBClassifier(base_score=None, booster=None, colsample_bylevel=None, <br>colsample_bynode=None, colsample_bytree=None, gamma=None, gpu_id=None, <br>importance_type='gain', interaction_constraints=None, learning_rate=None, <br>max_delta_step=None, max_depth=None, min_child_weight=None, missing=nan, <br>monotone_constraints=None, n_estimators=100, n_jobs=None, num_parallel_tree=None, <br>random_state=42, reg_alpha=None, reg_lambda=None, scale_pos_weight=None, <br>subsample=None, tree_method=None, validate_parameters=None, verbosity=None) | 0.990000 | 0.997000 | 0.826000 | 0.683000 | 141 |
| 13 | LogisticRegression(random_state=42, solver='liblinear') | 0.987000 | 0.992000 | 0.826000 | 0.734000 | 4 |
| 14 | XGBClassifier(base_score=None, booster=None, colsample_bylevel=None, <br>colsample_bynode=None, colsample_bytree=None, gamma=None, gpu_id=None, <br>importance_type='gain', interaction_constraints=None, learning_rate=None, <br>max_delta_step=None, max_depth=None, min_child_weight=None, missing=nan, <br>monotone_constraints=None, n_estimators=100, n_jobs=None, num_parallel_tree=None, <br>random_state=42, reg_alpha=None, reg_lambda=None, scale_pos_weight=None, subsample=None, <br>tree_method=None, validate_parameters=None, verbosity=None) | 0.992000 | 0.999000 | 0.825000 | 0.715000 | 114 |
| 15 | LogisticRegression(random_state=42, solver='liblinear') | 0.989000 | 0.993000 | 0.845000 | 0.708000 | 5 |
| 16 | XGBClassifier(base_score=None, booster=None, colsample_bylevel=None, <br>colsample_bynode=None, colsample_bytree=None, gamma=None, gpu_id=None, <br>importance_type='gain', interaction_constraints=None, learning_rate=None, <br>max_delta_step=None, max_depth=None, min_child_weight=None, missing=nan, <br>monotone_constraints=None, n_estimators=100, n_jobs=None, num_parallel_tree=None, <br>random_state=42, reg_alpha=None, reg_lambda=None, scale_pos_weight=None, subsample=None, <br>tree_method=None, validate_parameters=None, verbosity=None) | 0.990000 | 0.997000 | 0.829000 | 0.690000 | 118 |
| 17 | LogisticRegression(random_state=42, solver='liblinear') | 0.990000 | 0.994000 | 0.849000 | 0.725000 | 9 |
| 18 | XGBClassifier(base_score=None, booster=None, colsample_bylevel=None, <br>colsample_bynode=None, colsample_bytree=None, gamma=None, gpu_id=None, <br>importance_type='gain', interaction_constraints=None, learning_rate=None, <br>max_delta_step=None, max_depth=None, min_child_weight=None, missing=nan, <br>monotone_constraints=None, n_estimators=100, n_jobs=None, num_parallel_tree=None, <br>random_state=42, reg_alpha=None, reg_lambda=None, scale_pos_weight=None, subsample=None, <br>tree_method=None, validate_parameters=None, verbosity=None) | 0.991000 | 0.999000 | 0.843000 | 0.704000 | 129 |
|  |  |  |  |  |  |  |

## Data Dictionary 

| Feature | Type | Dataset | Description |
|---|---|---|---|
| Id | Integer | Test | The id of the record |
| Date | Object | Train/Test/Weather | Date that the WNV test is performed for Train/TestDate of weather measurement for Weather |
| Address | Object | Train/Test | Approximate address of the location of trap. This is used to send to the GeoCoder. |
| Species | Object | Train/Test | The species of mosquitos |
| Block | Integer | Train/Test | Block number of address |
| Street | Object | Train/Test | Street name |
| Trap | Object | Train/Test | Id of the trap |
| AddressNumberAndStreet | Object | Train/Test | Approximate address returned from GeoCoder |
| Latitude | Float | Train/Test/Spray | Latitude returned from GeoCoder for Train/Test <br>Latitude of the spray |
| Longitude | Float | Train/Test/Spray | Longitude returned from GeoCoder for Train/Test<br>Longitude of the spray |
| AddressAccuracy | Integer | Train/Test | Accuracy returned from GeoCoder |
| NumMosquitos | Integer | Train | Number of mosquitoes caught in this trap |
| WnvPresent | Integer | Train | Whether West Nile Virus was present in these mosquitos. <br>1 means WNV is present, and 0 means not present. |
| Station | Integer | Weather | 1; automated station without a precipitation descriminator. <br>2; automated station with precipitation descriminator. |
| Tmax | Integer | Weather | Maximum Temperature |
| Tmin | Integer | Weather | Minimum Temperature |
| Tavg | Integer | Weather | Average Temperature |
| Depart | Integer | Weather | Departure from normal |
| Dewpoint | Integer | Weather | Average dew point |
| Wetbulb | Integer | Weather | Average wet bulb |
| Heat | Integer | Weather | Heating (Season begins with July) |
| Cool | Integer | Weather | Cooling (Season begins with January) |
| Sunrise | Object | Weather | Sunrise Time (Calculated, not observed) |
| Sunset | Object | Weather | Sunset Time (Calculated, not observed) |
| CodeSum | Object | Weather | Significant Weather Types<br><br>+FC TORNADO/WATERSPOUT<br> FC  FUNNEL CLOUD<br> TS  THUNDERSTORM<br> GR  HAIL<br>RA RAIN<br>DZ DRIZZLE<br>SN SNOW<br>SG SNOW GRAINS<br>GS SMALL HAIL &/OR SNOW PELLETS<br>PL ICE PELLETS<br>IC ICE CRYSTALS<br>FG+ HEAVY FOG (FG & LE.25 MILES VISIBILITY) FG FOG<br>BR MIST<br>UP UNKNOWN PRECIPITATION<br>HZ HAZE<br>FU SMOKE<br>VA VOLCANIC ASH<br>DU WIDESPREAD DUST<br>DS DUSTSTORM<br>PO SAND/DUST WHIRLS<br>SA SAND<br>SS SANDSTORM<br>PY SPRAY<br>SQ SQUALL<br>DR LOW DRIFTING<br>SH SHOWER<br>FZ FREEZING<br>MI SHALLOW<br>PR PARTIAL<br>BC PATCHES<br>BL BLOWING<br>VC VICINITY<br>- LIGHT + HEAVY<br>"NO SIGN" MODERATE |
| Depth | Integer | Weather | Depth of snow/ice in inches SNOW/ICE (ON GROUND)(1200 UTC)T = TRACEM = MISSING DATA |
| Water1 | Integer | Weather | WATER EQUIVALENT (1800 UTC) M = MISSING DATA |
| SnowFall | Float | Weather | SNOWFALL (INCHES AND TENTHS)(2400 LST)*<br>T = TRACEM = MISSING DATA |
| PrecipTotal | Float | Weather | WATER EQUIVALENT(INCHES & HUNDREDTHS(2400 LST) RAINFALL & MELTED SNOW<br>M = MISSING DATAT = TRACE |
| StnPressure | Float | Weather | Pressure in inches |
| SeaLevel | Float | Weather | Average Sea Level Pressure |
| ResultSpeed | Float | Weather | Resultant Wind Speed |
| ResultDir | Integer | Weather | Resultant Wind Direction (Whole Degree) |
| AvgSpeed | Float | Weather | Wing Average Speed |
